# Calculate the reaction rates for the primary reactions, as well as the main secondary reactions

In [70]:
import numpy as np
from numpy import float64
from numpy.typing import NDArray

In [71]:
# <sigmav> formulas (from cfspopcon)

def sigmav_DT_Hively(ion_temp_profile: NDArray[float64]) -> NDArray[float64]:
    r"""Deuterium-Tritium reaction.

    Calculate :math:`\langle \sigma v \rangle` for a given characteristic ion energy.
    Formulation from table 1, column S5 in :cite:`hively_convenient_1977`.
    Curvefit was performed for the range of [1,80]keV.

    Args:
        ion_temp_profile: ion temperature profile [keV]

    Returns:
        :math:`\langle \sigma v \rangle` in cm^3/s.
    """
    A = [-21.377692, -25.204054, -7.1013427 * 1e-2, 1.9375451 * 1e-4, 4.9246592 * 1e-6, -3.9836572 * 1e-8]
    r = 0.2935
    sigmav = np.exp(
        A[0] / ion_temp_profile**r
        + A[1]
        + A[2] * ion_temp_profile
        + A[3] * ion_temp_profile**2.0
        + A[4] * ion_temp_profile**3.0
        + A[5] * ion_temp_profile**4.0
    )
    return sigmav  # type: ignore[no-any-return] # [cm^3/s]

def sigmav_DT_BoschHale(ion_temp_profile: NDArray[float64]) -> NDArray[float64]:
    r"""Deuterium-Tritium reaction.

    Calculate :math:`\langle \sigma v \rangle` product for a given characteristic ion energy using Bosch Hale method.

    :func:`sigmav_DT_BoschHale` is more accurate than :func:`sigmav_DT` for ion_temp_profile > ~48.45 keV (estimate based on
    linear interp between errors found at available datapoints).
    Maximum error = 1.4% within range 50-1000 keV from available NRL data.

    Formulation from :cite:`bosch_improved_1992`

    Args:
        ion_temp_profile: ion temperature profile [keV]

    Returns:
        :math:`\langle \sigma v \rangle` in cm^3/s.

    """
    # Bosch Hale coefficients for DT reaction
    C = [0.0, 1.173e-9, 1.514e-2, 7.519e-2, 4.606e-3, 1.35e-2, -1.068e-4, 1.366e-5]
    B_G = 34.3827
    mr_c2 = 1124656

    theta = ion_temp_profile / (
        1
        - (ion_temp_profile * (C[2] + ion_temp_profile * (C[4] + ion_temp_profile * C[6])))
        / (1 + ion_temp_profile * (C[3] + ion_temp_profile * (C[5] + ion_temp_profile * C[7])))
    )
    eta = (B_G**2 / (4 * theta)) ** (1 / 3)
    sigmav = C[1] * theta * np.sqrt(eta / (mr_c2 * ion_temp_profile**3)) * np.exp(-3 * eta)
    return sigmav  # type: ignore[no-any-return] # [cm^3/s]


def sigmav_DD_Hively(ion_temp_profile: NDArray[float64]) -> tuple[NDArray[float64], NDArray[float64], NDArray[float64]]:
    r"""Deuterium-Deuterium reaction.

    Calculate :math:`\langle \sigma v \rangle` for a given characteristic ion energy.
    Formulation from column S5, in table 3 and 4 in :cite:`hively_convenient_1977`.
    Curvefit was performed for the range of [1,80]keV.

    Args:
        ion_temp_profile: ion temperature profile [keV]

    Returns:
        :math:`\langle \sigma v \rangle` tuple (total, D(d,p)T, D(d,n)3He) in cm^3/s.
    """
    a_1 = [
        -15.511891,
        -35.318711,
        -1.2904737 * 1e-2,
        2.6797766 * 1e-4,
        -2.9198685 * 1e-6,
        1.2748415 * 1e-8,
    ]  # For D(d,p)T
    r_1 = 0.3735
    a_2 = [
        -15.993842,
        -35.017640,
        -1.3689787 * 1e-2,
        2.7089621 * 1e-4,
        -2.9441547 * 1e-6,
        1.2841202 * 1e-8,
    ]  # For D(d,n)3He
    r_2 = 0.3725
    # Ti in units of keV, sigmav in units of cm^3/s
    sigmav_1: NDArray[float64] = np.exp(
        a_1[0] / ion_temp_profile**r_1
        + a_1[1]
        + a_1[2] * ion_temp_profile
        + a_1[3] * ion_temp_profile**2.0
        + a_1[4] * ion_temp_profile**3.0
        + a_1[5] * ion_temp_profile**4.0
    )
    sigmav_2: NDArray[float64] = np.exp(
        a_2[0] / ion_temp_profile**r_2
        + a_2[1]
        + a_2[2] * ion_temp_profile
        + a_2[3] * ion_temp_profile**2.0
        + a_2[4] * ion_temp_profile**3.0
        + a_2[5] * ion_temp_profile**4.0
    )
    sigmav_tot: NDArray[float64] = sigmav_1 + sigmav_2
    return sigmav_tot, sigmav_1, sigmav_2  # [cm^3/s]


def sigmav_DD_BoschHale(ion_temp_profile: NDArray[float64]) -> tuple[NDArray[float64], NDArray[float64], NDArray[float64]]:
    r"""Deuterium-Deuterium reaction.

    Calculate :math:`\langle \sigma v \rangle` product for a given characteristic ion energy using Bosch Hale method.

    Function tested on available data at [1, 2, 5, 10, 20, 50, 100] keV.
    Maximum error = 3.8% within range 5-50 keV and increases significantly outside of [5, 50] keV.

    Uses DD cross section formulation from :cite:`bosch_improved_1992`.

    Other form in :cite:`langenbrunner_analytic_2017`.

    Args:
        ion_temp_profile: ion temperature profile [keV]

    Returns:
        :math:`\langle \sigma v \rangle` tuple (total, D(d,p)T, D(d,n)3He) in cm^3/s.
    """
    # For D(d,n)3He
    cBH_1 = [((31.3970**2) / 4.0) ** (1.0 / 3.0), 5.65718e-12, 3.41e-03, 1.99e-03, 0, 1.05e-05, 0, 0]  # 3.72e-16,

    mc2_1 = 937814.0

    # For D(d,p)T
    cBH_2 = [((31.3970**2) / 4.0) ** (1.0 / 3.0), 5.43360e-12, 5.86e-03, 7.68e-03, 0, -2.96e-06, 0, 0]  # 3.57e-16,

    mc2_2 = 937814.0

    thetaBH_1 = ion_temp_profile / (
        1
        - (
            (cBH_1[2] * ion_temp_profile + cBH_1[4] * ion_temp_profile**2 + cBH_1[6] * ion_temp_profile**3)
            / (1 + cBH_1[3] * ion_temp_profile + cBH_1[5] * ion_temp_profile**2 + cBH_1[7] * ion_temp_profile**3)
        )
    )

    thetaBH_2 = ion_temp_profile / (
        1
        - (
            (cBH_2[2] * ion_temp_profile + cBH_2[4] * ion_temp_profile**2 + cBH_2[6] * ion_temp_profile**3)
            / (1 + cBH_2[3] * ion_temp_profile + cBH_2[5] * ion_temp_profile**2 + cBH_2[7] * ion_temp_profile**3)
        )
    )

    etaBH_1: float = cBH_1[0] / (thetaBH_1 ** (1.0 / 3.0))
    etaBH_2: float = cBH_2[0] / (thetaBH_2 ** (1.0 / 3.0))

    sigmav_1: NDArray[float64] = cBH_1[1] * thetaBH_1 * np.sqrt(etaBH_1 / (mc2_1 * (ion_temp_profile**3.0))) * np.exp(-3.0 * etaBH_1)
    sigmav_2: NDArray[float64] = cBH_2[1] * thetaBH_2 * np.sqrt(etaBH_2 / (mc2_2 * (ion_temp_profile**3.0))) * np.exp(-3.0 * etaBH_2)
    sigmav_tot: NDArray[float64] = sigmav_1 + sigmav_2

    return sigmav_tot, sigmav_1, sigmav_2  # [cm^3/s]


def sigmav_DHe3_BoschHale(ion_temp_profile: NDArray[float64]) -> NDArray[float64]:
    r"""Deuterium-Helium-3 reaction.

    Calculate :math:`\langle \sigma v \rangle` for a given characteristic ion energy.

    Function tested on available data at [1, 2, 5, 10, 20, 50, 100] keV.
    Maximum error = 8.4% within range 2-100 keV and should not be used outside range [2, 100] keV.

    Uses DD cross section formulation :cite:`bosch_improved_1992`.

    Args:
        ion_temp_profile: ion temperature profile [keV]

    Returns:
        :math:`\langle \sigma v \rangle` in cm^3/s.
    """
    # For He3(d,p)4He
    cBH_1 = [
        ((68.7508**2) / 4.0) ** (1.0 / 3.0),
        5.51036e-10,  # 3.72e-16,
        6.41918e-03,
        -2.02896e-03,
        -1.91080e-05,
        1.35776e-04,
        0,
        0,
    ]

    mc2_1 = 1124572.0

    thetaBH_1 = ion_temp_profile / (
        1
        - (
            (cBH_1[2] * ion_temp_profile + cBH_1[4] * ion_temp_profile**2 + cBH_1[6] * ion_temp_profile**3)
            / (1 + cBH_1[3] * ion_temp_profile + cBH_1[5] * ion_temp_profile**2 + cBH_1[7] * ion_temp_profile**3.0)
        )
    )

    etaBH_1: float = cBH_1[0] / (thetaBH_1 ** (1.0 / 3.0))

    sigmav: NDArray[float64] = cBH_1[1] * thetaBH_1 * np.sqrt(etaBH_1 / (mc2_1 * (ion_temp_profile**3.0))) * np.exp(-3.0 * etaBH_1)

    return sigmav  # [cm^3/s]


In [72]:
def volumetric_reaction_rates_DDfuel(
    density_D: NDArray[float64], # [m^-3]
    temperature: NDArray[float64], # [keV]
    tau_p_T: NDArray[float64] = 1, # [s] needed to estimate the reaction rates of the secondary reaction D-T
    tau_p_He3: NDArray[float64] = 1 # [s] needed to estimate the reaction rates of the secondary reaction D-He3
):
    
    DEBUG = False
    r"""Calculate volumetric reaction rates for D-D main reactions,
        as well as D-T and D-He3 secondary reactions.

    Args:
        density_D: D density [m^-3]
        temperature: ion temperature profile [keV]
        tau_p_T: tritium confinement time [s]           
        tau_p_He3: He3 confinement time [s]
    Returns:
        dictionary: dictionary with the following keys:
            - rr_DDp: volumetric reaction rate for D-D -> D-T + p [1/m^3/s]
            - rr_DDn: volumetric reaction rate for D-D -> D-He3 + n [1/m^3/s]
            - rr_DT: volumetric reaction rate for D-T -> He4 + n [1/m^3/s]
            - rr_DHe3: volumetric reaction rate for D-He3 -> He4 + p [1/m^3/s]
            - rr_tot: total volumetric reaction rate [1/m^3/s]
            - density_T: tritium density [m^-3]
            - density_He3: He3 density [m^-3]
            - prob_DDp: probability of D-D -> D-T + p [-]
            - prob_DDn: probability of D-D -> D-He3 + n [-]
            - prob_DT: probability of D-T -> He4 + n [-]
            - prob_DHe3: probability of D-He3 -> He4 + p [-]
            - prob_tot: total probability [-]
    """
    
    
    """ recalling that
            |----> He3(1.01 MeV) + n(2.45 MeV)
    D + D ->|
            |----> T(1.01 MeV) + p(3.02 MeV)
            
    AND
    Following the calculations done by Pedretti, E., Rollet, S., 
    "COMPARISON OF NEUTRON PRODUCTIONS DURING OPERATION OF THE ARIES-III SECOND STABILITY D3 He TOKAMAK REACTOR WITHOUT AND WITH TRITIUM-ASSISTED STARTUP" 
    (1994):

    Production of deuteric T = (n_D/2)^2 * < sigmav >_DDp  

    Losses of deuteric T = n_D * n_T,D * < sigmav >_DT + n_T,D / tau_p
    (where n_T,D is the tritium produced bby DDp reactions)

    At steady-state the two equilibrate:  
    (1/2) * n_D^2 * < sigmav >_DDp = n_D * n_T,D * < sigmav >_DT + n_T,D / tau_p  

    Then:  
    n_T,D =( (1/2) * n_D^2 * < sigmav >_DDp ) / ( n_D * < sigmav >_DT + 1 / tau_p )
    
    A similar discussion can be made for the He3 production
    """
    
    
    # calculate the reactivities for all the main and secondary reactions
    # use BoschHale fit exclusively
    sigmav_DD_tot = sigmav_DD_BoschHale(temperature)[0]*1e-6  # [m^3/s]
    sigmav_DD_p = sigmav_DD_BoschHale(temperature)[1]*1e-6  # [m^3/s]
    sigmav_DD_n = sigmav_DD_BoschHale(temperature)[2]*1e-6  # [m^3/s]
    sigmav_DT = sigmav_DT_BoschHale(temperature)*1e-6  # [m^3/s]
    sigmav_DHe3 = sigmav_DHe3_BoschHale(temperature)*1e-6  # [m^3/s]
    if DEBUG == True:
        print(f"sigmav in [m^3/s]:\n",
            f"    sigmav_DD_tot: {sigmav_DD_tot}, sigmav_DD_p: {sigmav_DD_p}, sigmav_DD_n: {sigmav_DD_n}, sigmav_DT: {sigmav_DT}, sigmav_DHe3: {sigmav_DHe3}")
    
    # calculate volumetric reaction rates for the two main reactions (the 0.5 factor takes into account fusion from identical particles)
    rr_DDp = 0.5 * sigmav_DD_p * density_D**2 # [1/m^3/s]
    rr_DDn = 0.5 * sigmav_DD_n * density_D**2 # [1/m^3/s]
    
    # estimate volumetric reaction rates for the secondary reactions    
    density_T = (0.5 * density_D**2 * sigmav_DD_p)/(density_D * sigmav_DT + 1/tau_p_T) # [m^-3]
    density_He3 = (0.5 * density_D**2 * sigmav_DD_n)/(density_D * sigmav_DHe3 + 1/tau_p_He3) # [m^-3]
    
    rr_DT = sigmav_DT * density_D * density_T # [1/m^3/s]
    rr_DHe3 = sigmav_DHe3 * density_D * density_He3 # [1/m^3/s]
    
    # calculate the probabilities associated to the reactions
    rr_tot = rr_DDp + rr_DDn + rr_DT + rr_DHe3 # [1/m^3/s]
    prob_DDp = rr_DDp / rr_tot
    prob_DDn = rr_DDn / rr_tot
    prob_DT = rr_DT / rr_tot
    prob_DHe3 = rr_DHe3 / rr_tot
    
    # build a dictionary with the results
    dictionary = {
        "rr_DDp": rr_DDp, # [1/m^3/s]
        "rr_DDn": rr_DDn, # [1/m^3/s]
        "rr_DT": rr_DT, # [1/m^3/s]
        "rr_DHe3": rr_DHe3, # [1/m^3/s]
        "rr_tot": rr_tot,   # [1/m^3/s]
        "density_T": density_T, # [m^-3]
        "density_He3": density_He3, # [m^-3]
        "prob_DDp": prob_DDp,   # [-]
        "prob_DDn": prob_DDn,   # [-]
        "prob_DT": prob_DT,  # [-]
        "prob_DHe3": prob_DHe3, # [-]
        "prob_tot": prob_DDp + prob_DDn + prob_DT + prob_DHe3 # [-]
    }
    if DEBUG == True:
        print(f"reaction rates in [1/m^3/s]:\n",
            f"    rr_DDp: {rr_DDp}, rr_DDn: {rr_DDn}, rr_DT: {rr_DT}, rr_DHe3: {rr_DHe3}, rr_tot: {rr_tot}")
        print(f"density in [m^-3]:\n",
            f"    density_T: {density_T}, density_He3: {density_He3}")
        print(f"probabilities:\n",
            f"    prob_DDp: {prob_DDp}, prob_DDn: {prob_DDn}, prob_DT: {prob_DT}, prob_DHe3: {prob_DHe3}, prob_tot: {prob_DDp + prob_DDn + prob_DT + prob_DHe3}")
        
    return dictionary
    

# Calculate startup time and net Power

In [73]:
# Input Data
V_plasma = 150 # m^3 Plasma volume
n_e_avg = 1.7e20 # 1/m^3 Average electron density
Pf_DT = 1000 # MW Fusion power for DT reactor
T_e_avg = 14 # keV Average electron temperature
P_aux = 100 # MW Auxiliary heating power
I_ST = 3 # kg Inventory target

tau_p_T = 2 # s Tritium confinement time
tau_p_He3 = 2 # s He3 confinement time

n_D = n_e_avg # since only D is present in the plasma at the beginning

Li6_enrichment_fraction = 0.5 # Fraction of Li-6 in the breeding blanket

#define Tritium breeding ratios for DT and DD neutrons
TBR = 1.15
TBR_DDn = 1

In [81]:
# Calculate the reaction rates
DD_reaction_rates = volumetric_reaction_rates_DDfuel(n_D, T_e_avg, tau_p_T, tau_p_He3)
print("Reaction Rates:")
print(DD_reaction_rates)

Reaction Rates:
{'rr_DDp': 1.7461585412408442e+16, 'rr_DDn': 1.8533333182477384e+16, 'rr_DT': 1320129572598525.5, 'rr_DHe3': 5621022626666.936, 'rr_tot': 3.7320669190111016e+16, 'density_T': 3.2282911679619828e+16, 'density_He3': 3.705542431970143e+16, 'prob_DDp': 0.46787975112287905, 'prob_DDn': 0.4965970220970267, 'prob_DT': 0.03537261258295778, 'prob_DHe3': 0.00015061419713653895, 'prob_tot': 1.0}


In [78]:
# calculate the tritium production rates
Tdot_fusion = DD_reaction_rates["rr_DDp"] * V_plasma - DD_reaction_rates["rr_DT"] * V_plasma # [1/s] is the effective rate of tritium production due to DD fusion, considering the losses due to DT neutrons
Tdot_breedingDT = TBR * DD_reaction_rates["rr_DT"] * V_plasma # [1/s] is the rate of tritium production due to DT neutrons interacting with the breeding blanket (assuming each neutron produces 1 tritium)
Tdot_breedingDD = TBR_DDn * DD_reaction_rates["rr_DDn"] * V_plasma * Li6_enrichment_fraction # [1/s] is the rate of tritium production due to DD neutrons interacting with the Li6 in the breeding blanket (assuming each neutron produces 1 tritium)
Tdot_transport = DD_reaction_rates["density_T"]/tau_p_T # [1/s] is the rate of tritium exiting the plasma due to transport

Tdot_tot = Tdot_fusion + Tdot_breedingDT + Tdot_breedingDD + Tdot_transport # [1/s] is the total rate of tritium production in the system
print(f"total rate of tritium production in the system: {Tdot_tot:.2e} 1/s")

# Calculate the startup time
N_ST = I_ST*1000/6.03*6.022e23 # [-] is the number of tritium atoms needed for startup (I_ST[g]/6.03[g/mol]*6.022e23[atoms/mol])
startup_time = N_ST / (Tdot_tot) # [s] is the time needed to produce the required amount of tritium for startup
print(f"Startup time: {startup_time:.2f} s = {startup_time/3600:.2f} h = {startup_time/3600/24:.2f} days = {startup_time/3600/24/365:.2f} years")


total rate of tritium production in the system: 4.06e+18 1/s
Startup time: 73883087.29 s = 20523.08 h = 855.13 days = 2.34 years


In [76]:
# Calculate the Reactor Power for DD reactions
E_DDp = 4.03*1e6*1.6e-19 # [J] energy released by DDp reactions
E_DDn = 3.46*1e6*1.6e-19 # [J] energy released by DDn reactions
E_DT = 17.6*1e6*1.6e-19 # [J] energy released by DT reactions
E_DHe3 = 18.0153*1e6*1.6e-19 # [J] energy released by DHe3 reactions

Pf_DD = DD_reaction_rates["rr_DDp"]*E_DDp*V_plasma + DD_reaction_rates["rr_DDn"]*E_DDn*V_plasma # [W] is the power produced by DD reactions
Pf_DD_DT = DD_reaction_rates["rr_DT"]*E_DT*V_plasma # [W] is the power produced by DT sub-reactions
Pf_DD_DHe3 = DD_reaction_rates["rr_DHe3"]*E_DHe3*V_plasma # [W] is the power produced by DHe3 sub-reactions
Pf_DD_tot = Pf_DD + Pf_DD_DT + Pf_DD_DHe3 # [W] is the total power produced in a DD reactor
print(f"Pf_DD_tot: {Pf_DD_tot/1e6:.2f} MW, of which {Pf_DD/1e6:.2f} MW is from DD reactions, {Pf_DD_DT/1e6:.2f} MW is from DT sub-reactions and {Pf_DD_DHe3/1e6:.2f} MW is from DHe3 sub-reactions")
Pf_DT = 1/4*n_e_avg**2*sigmav_DT_BoschHale(T_e_avg)*1e-6*E_DT*V_plasma # [W] is the power produced by DT reactions
print(f"Pf_DT: {Pf_DT/1e6:.2f} MW")

def calculate_P_e_net(Pf:float, Q:float, eta_th:float) -> float:
    r"""Calculate the net electrical power produced by the reactor.

    Args:
        Pf: Fusion power [W]
        Q: Auxiliary heating power [W]
        eta_th: Thermal efficiency of the reactor [-]

    Returns:
        P_e_net: Net electrical power produced by the reactor [W]
    """
    # Q = (Pf - Paux)/Paux then Paux = Pf/(Q+1)
    P_aux = Pf / (Q + 1) # [W] is the auxiliary heating power needed to maintain the plasma temperature
    P_e_net = eta_th * Pf - P_aux # [W] is the net electrical power produced by the reactor
    return P_e_net
eta_th = 0.3 # [-] is the thermal efficiency of the reactor
Q = 10

P_e_net_DD = calculate_P_e_net(Pf_DD, Q, eta_th) # [W] is the net electrical power produced by the reactor
P_e_net_DT = calculate_P_e_net(Pf_DT, Q, eta_th) # [W] is the net electrical power produced by the reactor
print(f"P_e_net_DD: {P_e_net_DD/1e6:.2f} MW")
print(f"P_e_net_DT: {P_e_net_DT/1e6:.2f} MW")

E_lost = np.abs(P_e_net_DD*startup_time - P_e_net_DT*startup_time) # [J] is the energy lost during the startup time operation in DD
print(f"E_lost: {E_lost/1e6:.2f} MJ")

Cost_per_MWh = 0.25 # [USD] is the cost of electricity
J_to_MWh = 2.777777777E-10 # [MWh] is the conversion factor from Joules to MWh
Dollar_lost = Cost_per_MWh * E_lost * J_to_MWh # [USD] is the cost of the lost energy during the startup time operation in DD
print(f"$ lost: {Dollar_lost:.2f} USD")

Pf_DD_tot: 3.79 MW, of which 3.23 MW is from DD reactions, 0.56 MW is from DT sub-reactions and 0.00 MW is from DHe3 sub-reactions
Pf_DT: 734.10 MW
P_e_net_DD: 0.67 MW
P_e_net_DT: 153.49 MW
E_lost: 11290756920.16 MJ
$ lost: 784080.34 USD
